In [6]:
import numpy as np
import pandas as pd
from Recommendation import collaborative_filtering

In [16]:
import os
filedir = os.path.dirname(os.path.realpath('__file__'))

In [17]:
beer_data = pd.read_csv(filedir + r"\Data\beer_reviews\beer_reviews.csv")
beer_data.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [18]:
beer_data.isnull().sum()

brewery_id                0
brewery_name             15
review_time               0
review_overall            0
review_aroma              0
review_appearance         0
review_profilename      348
beer_style                0
review_palate             0
review_taste              0
beer_name                 0
beer_abv              67785
beer_beerid               0
dtype: int64

In [19]:
print beer_data.shape[0]

1586614


In [20]:
reviewers = beer_data.groupby(beer_data.review_profilename)["review_overall"].count()
reviewers = reviewers[reviewers > 1000].index
len(reviewers)

252

In [21]:
demo_data = beer_data[beer_data["review_profilename"].isin(reviewers)]
demo_data.shape

(413769, 13)

In [22]:
req_beers = demo_data.groupby(demo_data.beer_name)["review_overall"].count()
req_beers = req_beers[req_beers > 200].index
len(req_beers)

70

In [23]:
demo_data = demo_data[demo_data.beer_name.isin(req_beers)]
demo_data.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
1559,11715,Destiny Brewing Company,1137124057,4.0,3.5,4.0,blitheringidiot,American Pale Ale (APA),3.5,3.5,Pale Ale,4.5,26420
1560,11715,Destiny Brewing Company,1129504403,4.0,2.5,4.0,NeroFiddled,American Pale Ale (APA),4.0,3.5,Pale Ale,4.5,26420
1563,11715,Destiny Brewing Company,1137125989,3.5,3.0,4.0,blitheringidiot,American IPA,4.0,4.0,IPA,NaN,26132
1564,11715,Destiny Brewing Company,1130936611,3.0,3.0,3.0,Gavage,American IPA,4.0,3.5,IPA,NaN,26132
1565,11715,Destiny Brewing Company,1129505059,4.5,3.5,3.5,NeroFiddled,American IPA,4.0,4.0,IPA,NaN,26132


In [24]:
data = demo_data[["review_profilename","beer_name","review_overall"]]
data.head()

,review_profilename,beer_name,review_overall
1559,blitheringidiot,Pale Ale,4.0
1560,NeroFiddled,Pale Ale,4.0
1563,blitheringidiot,IPA,3.5
1564,Gavage,IPA,3.0
1565,NeroFiddled,IPA,4.5


In [25]:
profiles, beers = np.unique(data.review_profilename), np.unique(data.beer_name)
hash_profiles = {profiles[i] : i for i in range(len(profiles))}
hash_beers = {beers[i] : i for i in range(len(beers))}
hash_profiles_inv = {v:k for k,v in hash_profiles.items()}
hash_beers_inv = {v:k for k,v in hash_beers.items()}
# Constructing Utility Matrix
utility_matrix = np.zeros((len(profiles), len(beers)))

In [12]:
for i in range(data.shape[0]):
    utility_matrix[hash_profiles[data.iloc[i,0]], hash_beers[data.iloc[i,1]]] = data.iloc[i,2]

In [13]:
utility_matrix

array([[ 4. ,  4. ,  4. , ...,  4. ,  4.5,  4.5],
       [ 4. ,  3.5,  5. , ...,  4.5,  4. ,  4. ],
       [ 0. ,  4. ,  4.5, ...,  4.5,  4.5,  2.5],
       ..., 
       [ 4.5,  4.5,  0. , ...,  5. ,  4.5,  4. ],
       [ 3.5,  5. ,  3.5, ...,  5. ,  5. ,  4. ],
       [ 4.5,  5. ,  5. , ...,  4. ,  4.5,  4. ]])

In [14]:
import collaborative_filtering

In [15]:
rec_col_ii_cos = collaborative_filtering.collaborative_filtering(utility_matrix)
rec_col_ii_cos.similarity_mat()
rec_col_ii_cos.final_recommendation_score_matrix()

array([[ 188.07111282,  193.07049506,  176.63535134, ...,  188.88930557,
         187.63589341,  181.42991947],
       [ 158.08530597,  162.55565108,  149.33204249, ...,  159.04124466,
         158.07778001,  152.4447081 ],
       [ 172.47375733,  178.16041259,  164.96691182, ...,  175.59812799,
         173.18875169,  165.76589739],
       ..., 
       [ 164.76483802,  168.91268119,  153.46045938, ...,  165.1336547 ,
         164.53757226,  159.07083299],
       [ 175.65952119,  181.36101007,  166.46438455, ...,  177.74712273,
         176.47037099,  169.52517439],
       [ 253.28940577,  260.54287353,  239.84256603, ...,  255.86766501,
         252.89847848,  243.5028803 ]])

In [16]:
rec_col_ii_cos.output_recommendations("Reviewer Name", "Beer Name", hash_profiles_inv, hash_beers_inv).head()

,Reviewer Name,Beer Name,Recommended_score,Rank,Flag
0,ADR,90 Minute IPA,193.07,1,1
1,ADR,Stone IPA (India Pale Ale),192.319,2,0
2,ADR,Storm King Stout,191.876,3,1
3,ADR,Chocolate Stout,191.487,4,1
4,ADR,Old Rasputin Russian Imperial Stout,190.794,5,1


In [17]:
rec_col_ii_jac = collaborative_filtering.collaborative_filtering(utility_matrix, product = "jac")
rec_col_ii_jac.similarity_mat()
rec_col_ii_jac.final_recommendation_score_matrix()

array([[ 135.08328168,  138.52037461,  125.78278454, ...,  139.95961871,
         138.75277425,  121.22910523],
       [ 115.10063407,  118.23991407,  108.07338076, ...,  119.83358815,
         118.48004727,  102.99161938],
       [ 123.23781384,  128.35826598,  118.84521278, ...,  131.75147282,
         129.25744928,  109.67761896],
       ..., 
       [ 118.43019569,  120.79164888,  107.96716656, ...,  122.18219241,
         121.81364169,  106.14453681],
       [ 127.58557167,  132.17896798,  120.36112267, ...,  134.59426743,
         133.02831909,  114.06629185],
       [ 181.03719705,  185.98155859,  170.68997488, ...,  189.45316778,
         186.42647531,  160.9502921 ]])

In [18]:
rec_col_ii_jac.similarity_matrix

array([[ 1.        ,  0.72131148,  0.62235915, ...,  0.69041561,
         0.67932489,  0.62098873],
       [ 0.72131148,  1.        ,  0.62648557, ...,  0.73613445,
         0.70214521,  0.59967051],
       [ 0.62235915,  0.62648557,  1.        , ...,  0.74475843,
         0.63054608,  0.52860803],
       ..., 
       [ 0.69041561,  0.73613445,  0.74475843, ...,  1.        ,
         0.72757754,  0.58367347],
       [ 0.67932489,  0.70214521,  0.63054608, ...,  0.72757754,
         1.        ,  0.60880399],
       [ 0.62098873,  0.59967051,  0.52860803, ...,  0.58367347,
         0.60880399,  1.        ]])

In [19]:
rec_knn = collaborative_filtering.collaborative_filtering(utility_matrix, algo = "knn")
rec_knn.similarity_mat()
rec_knn.final_recommendation_score_matrix()

array([[  764.61571706,   854.4339317 ,   946.18330869, ...,
          875.73071106,   850.26708324,   683.78821663],
       [  383.66192168,   346.37276489,   841.64839765, ...,
          481.83164734,   520.65709583,   409.40277143],
       [  966.44526334,   793.7586809 ,   714.33998821, ...,
          817.00106825,   831.52016533,  1048.52246299],
       ..., 
       [ 1039.77460347,  1157.59635032,  1597.43042507, ...,
         1206.99538071,  1368.72361822,   929.15123322],
       [  555.30344614,   520.4035299 ,   811.44490018, ...,
          545.82810651,   189.9263015 ,   796.95356761],
       [ 1631.81771235,  1734.8901675 ,  1837.42680647, ...,
         1752.71973501,  1844.38670886,  1741.86680925]])

In [20]:
rec_knn.similarity_matrix

array([[  0.        ,  24.78911051,  36.1282438 , ...,  30.79772719,
         31.60300619,  29.89983278],
       [ 24.78911051,   0.        ,  33.55219814, ...,  24.84954728,
         27.33587387,  29.79093822],
       [ 36.1282438 ,  33.55219814,   0.        , ...,  28.42094298,
         38.53569774,  40.72775466],
       ..., 
       [ 30.79772719,  24.84954728,  28.42094298, ...,   0.        ,
         30.90711892,  36.44173432],
       [ 31.60300619,  27.33587387,  38.53569774, ...,  30.90711892,
          0.        ,  34.63740752],
       [ 29.89983278,  29.79093822,  40.72775466, ...,  36.44173432,
         34.63740752,   0.        ]])

In [21]:
df_cos_sim = pd.DataFrame(rec_col_ii_cos.similarity_matrix)
df_knn_sim = pd.DataFrame(rec_knn.similarity_matrix)
df_jac_sim = pd.DataFrame(rec_col_ii_jac.similarity_matrix)
df_cos_sim.head()

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,1.000000,0.922749,0.825934,0.849301,0.905678,0.894862,0.851388,0.805787,0.900097,0.867151,...,0.830264,0.916703,0.837809,0.855383,0.843675,0.880978,0.788307,0.884646,0.881187,0.880071
1,0.922749,1.000000,0.855639,0.853145,0.926908,0.925470,0.884737,0.845741,0.916541,0.885192,...,0.874303,0.938055,0.862350,0.908380,0.883119,0.889722,0.836458,0.926492,0.912770,0.886015
2,0.825934,0.855639,1.000000,0.742647,0.820316,0.810945,0.869941,0.862420,0.858410,0.782643,...,0.801935,0.860812,0.830305,0.825399,0.781827,0.789088,0.829841,0.901212,0.819715,0.771055
3,0.849301,0.853145,0.742647,1.000000,0.846363,0.857015,0.790998,0.746588,0.820601,0.827964,...,0.767482,0.831049,0.758495,0.823453,0.827750,0.821909,0.720159,0.816405,0.847466,0.836033
4,0.905678,0.926908,0.820316,0.846363,1.000000,0.903866,0.864828,0.809523,0.906262,0.876898,...,0.846017,0.900820,0.820203,0.886753,0.874747,0.884994,0.786476,0.887249,0.870612,0.870816


In [ ]:
import cPickle as pickle

In [ ]:
with open(r"C:\Users\SHIVAM MAHAJAN\Desktop\Github\Recommendation-System\Data\beer_reviews\cossine_arm.pickle", "wb") as handle:
    pickle.dump(df_cos_sim, handle, protocol = pickle.HIGHEST_PROTOCOL)
with open(r"C:\Users\SHIVAM MAHAJAN\Desktop\Github\Recommendation-System\Data\beer_reviews\cossine_arm.pickle", "rb") as handle:
    check_df = pickle.load(handle)

In [ ]:
with open(r"C:\Users\SHIVAM MAHAJAN\Desktop\Github\Recommendation-System\Data\beer_reviews\jac_arm.pickle", "wb") as handle:
    pickle.dump(df_jac_sim, handle, protocol = pickle.HIGHEST_PROTOCOL)
with open(r"C:\Users\SHIVAM MAHAJAN\Desktop\Github\Recommendation-System\Data\beer_reviews\knn_arm.pickle", "wb") as handle:
    pickle.dump(df_knn_sim, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [80]:
with open(r"C:\Users\SHIVAM MAHAJAN\Desktop\Github\Recommendation-System\Data\beer_profile_hash.pickle", "wb") as handle:
    pickle.dump(hash_beers, handle)
    pickle.dump(hash_profiles, handle)
    pickle.dump(hash_beers_inv, handle)
    pickle.dump(hash_profiles_inv, handle)